# Building a co-author graph

In this notebook we're going to build an inferred graph of co-authors based on people collaborating on the same papers. We're also going to store a property on the relationship indicating the year of their first collaboration.

In [11]:
from neo4j import GraphDatabase

In [12]:
bolt_uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(bolt_uri, auth=("neo4j", "neo4jneo4j"))

print(driver.address)

localhost:7687


We can create the co-author graph by running the query below to do this:

In [13]:
query = """
CALL apoc.periodic.iterate(
  "MATCH (a1)<-[:AUTHOR]-(paper)-[:AUTHOR]->(a2:Author)
   WITH a1, a2, paper
   ORDER BY a1, paper.year
   RETURN a1, a2, collect(paper)[0].year AS year, count(*) AS collaborations",
  "MERGE (a1)-[coauthor:CO_AUTHOR {year: year}]-(a2)
   SET coauthor.collaborations = collaborations",
  {batchSize: 100})
"""

with driver.session(database="demo") as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=299 total=29890 timeTaken=58 committedOperations=29890 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 299, 'committed': 299, 'failed': 0, 'errors': {}} operations={'total': 29890, 'committed': 29890, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 0, 'relationshipsCreated': 14945, 'nodesCreated': 0, 'propertiesSet': 44874, 'relationshipsDeleted': 0, 'labelsRemoved': 0}>


Now that we've created our co-author graph, we want to come up with an approach that will allow us to predict future links (relationships) that will be created between people.